In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
def seq2onehot(seq):     #convert the cRBS sequences to one-hot encoding
    module = np.array([[[1,0,0,0]],
                       [[0,1,0,0]],
                       [[0,0,1,0]],
                       [[0,0,0,1]]])
    i = 0
    cRBS_onehot = []
    for i in seq:
        tmp = []
        for item in i:
            if item == 't' or item == 'T':
                tmp.append(module[0])
            elif item == 'c' or item == 'C':
                tmp.append(module[1])
            elif item == 'g' or item == 'G':
                tmp.append(module[2])
            elif item == 'a' or item == 'A':
                tmp.append(module[3])
            else:
                tmp.append([[0,0,0,0]])
        cRBS_onehot.append(tmp)
    cRBS_onehot=np.array(cRBS_onehot).astype('float32')
    return cRBS_onehot

In [3]:
x_train=np.load("./seq/npy/cRBS_long_N_0_1129.npy",allow_pickle=True)
x_train=np.load("./seq/cRBS_long_N_20_1129.npy",allow_pickle=True)
x_train=np.load("./seq/cRBS_long_R_0_1202.npy",allow_pickle=True)
x_train=np.load("./seq/cRBS_long_R_20_1202.npy",allow_pickle=True)
x_train=seq2onehot(x_train)
y_train=np.array(list(map(float,np.load("./seq/Fluorescence_intensity_N_0_1129.npy",allow_pickle=True))))
y_train=np.array(list(map(float,np.load("./seq/Fluorescence_intensity_N_20_1129.npy",allow_pickle=True))))
y_train=np.array(list(map(float,np.load("./seq/Fluorescence_intensity_R_0_1202.npy",allow_pickle=True))))
y_train=np.array(list(map(float,np.load("./seq/Fluorescence_intensity_R_20_1202.npy",allow_pickle=True))))
y_train_new=np.zeros((len(y_train),))
i=0
while i < len(y_train):
    y_train_new[i]=float(y_train[i])
    i=i+1
y_train=np.log2(y_train_new)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras.layers.core import Activation
from keras.layers import LeakyReLU,BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
model = Sequential()

model.add(
        #(6,1,4)
        Conv2D(100, (6, 1),
        padding='same',
        input_shape=(50, 1, 4))
        )
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(200, (5, 1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(200, (5, 1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dense(1024))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.4))
model.add(Dense(1))

In [5]:
train_feature=x_train[0:6000]
test_feature=x_train[6000:len(x_train)]
train_label=y_train[0:6000]
test_label=y_train[6000:len(x_train)]

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.fit(train_feature,train_label,epochs=1000,batch_size=64,validation_split=0.1,shuffle=True)#the codition of 0 g/l glucarate
#model.fit(train_feature,train_label,epochs=1000,batch_size=128,validation_split=0.1,shuffle=True)#the codition of 20 g/l glucarate

Epoch 1/1000
85/85 [==============================] - 1s 8ms/step - loss: 3.6569 - val_loss: 0.7840
Epoch 2/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8403 - val_loss: 0.7775
Epoch 3/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8292 - val_loss: 0.7763
Epoch 4/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8158 - val_loss: 0.7700
Epoch 5/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8076 - val_loss: 0.7649
Epoch 6/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8098 - val_loss: 0.7618
Epoch 7/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.8086 - val_loss: 0.7585
Epoch 8/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.7973 - val_loss: 0.7550
Epoch 9/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.7978 - val_loss: 0.7524
Epoch 10/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.7843 - val_loss: 0.7514

85/85 [==============================] - 0s 5ms/step - loss: 0.6159 - val_loss: 0.6031
Epoch 163/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6209 - val_loss: 0.6027
Epoch 164/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6179 - val_loss: 0.6034
Epoch 165/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6213 - val_loss: 0.6049
Epoch 166/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6241 - val_loss: 0.6003
Epoch 167/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6137 - val_loss: 0.6160
Epoch 168/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6166 - val_loss: 0.6020
Epoch 169/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6141 - val_loss: 0.6018
Epoch 170/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6142 - val_loss: 0.5997
Epoch 171/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.6136 - val_loss: 0.

85/85 [==============================] - 0s 5ms/step - loss: 0.5520 - val_loss: 0.5362
Epoch 323/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5487 - val_loss: 0.5347
Epoch 324/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5504 - val_loss: 0.5290
Epoch 325/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5551 - val_loss: 0.5287
Epoch 326/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5544 - val_loss: 0.5348
Epoch 327/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5518 - val_loss: 0.5352
Epoch 328/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5539 - val_loss: 0.5278
Epoch 329/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5510 - val_loss: 0.5280
Epoch 330/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5438 - val_loss: 0.5269
Epoch 331/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5446 - val_loss: 0.

85/85 [==============================] - 0s 5ms/step - loss: 0.5147 - val_loss: 0.5034
Epoch 483/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5216 - val_loss: 0.5069
Epoch 484/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5104 - val_loss: 0.5023
Epoch 485/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5181 - val_loss: 0.5034
Epoch 486/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5126 - val_loss: 0.5045
Epoch 487/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5168 - val_loss: 0.5055
Epoch 488/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5136 - val_loss: 0.5113
Epoch 489/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5112 - val_loss: 0.5052
Epoch 490/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5134 - val_loss: 0.5151
Epoch 491/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.5107 - val_loss: 0.

85/85 [==============================] - 0s 5ms/step - loss: 0.4984 - val_loss: 0.5036
Epoch 643/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4894 - val_loss: 0.4977
Epoch 644/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4986 - val_loss: 0.4979
Epoch 645/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4987 - val_loss: 0.4986
Epoch 646/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4973 - val_loss: 0.4972
Epoch 647/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4953 - val_loss: 0.4970
Epoch 648/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4946 - val_loss: 0.4988
Epoch 649/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4950 - val_loss: 0.5059
Epoch 650/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4918 - val_loss: 0.5011
Epoch 651/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4961 - val_loss: 0.

85/85 [==============================] - 0s 5ms/step - loss: 0.4825 - val_loss: 0.4967
Epoch 803/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4831 - val_loss: 0.5050
Epoch 804/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4887 - val_loss: 0.5070
Epoch 805/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4845 - val_loss: 0.4921
Epoch 806/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4843 - val_loss: 0.4905
Epoch 807/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4769 - val_loss: 0.4931
Epoch 808/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4801 - val_loss: 0.4915
Epoch 809/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4839 - val_loss: 0.4919
Epoch 810/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4859 - val_loss: 0.4927
Epoch 811/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4846 - val_loss: 0.

85/85 [==============================] - 0s 5ms/step - loss: 0.4775 - val_loss: 0.4929
Epoch 963/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4771 - val_loss: 0.4901
Epoch 964/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4719 - val_loss: 0.4919
Epoch 965/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4779 - val_loss: 0.4964
Epoch 966/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4702 - val_loss: 0.5033
Epoch 967/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4756 - val_loss: 0.4894
Epoch 968/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4703 - val_loss: 0.5018
Epoch 969/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4753 - val_loss: 0.5029
Epoch 970/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4686 - val_loss: 0.5021
Epoch 971/1000
85/85 [==============================] - 0s 5ms/step - loss: 0.4717 - val_loss: 0.

In [6]:
result=model.predict(test_feature,verbose=1)

33/33 [==============================] - 0s 3ms/step


In [7]:
import numpy as np
np.set_printoptions(threshold=np.inf)
result=result[:,0] 
print(result)
print(test_label)

[3.9097137 4.8050675 4.5737033 4.200033  5.009064  5.051808  4.486925
 4.1440907 3.8971324 4.598137  3.050656  4.4448586 5.132726  4.219309
 3.6298153 4.058346  4.0937967 4.4437923 4.6222167 4.2776074 4.8524456
 5.0075083 4.727906  4.634418  4.0589485 5.1802373 4.579729  3.8232203
 4.62611   3.8954258 3.6170902 4.4607544 5.1904235 4.500841  3.632389
 4.9425592 4.181675  4.149253  3.8856382 4.488966  4.569004  4.1939297
 3.614048  4.514145  3.7245884 5.503808  4.891088  4.6290884 4.1373043
 3.932416  2.8608487 4.399407  4.497065  4.7921295 4.2027493 4.6636343
 4.455542  3.3189735 4.70568   4.092537  4.539528  4.12221   3.9989603
 5.024825  4.2372265 5.329662  4.961679  4.267168  3.903556  4.0503073
 4.2619667 4.7409897 4.21918   4.879491  4.515703  4.7580314 4.37386
 4.8476753 4.498332  5.2443643 4.7861786 3.683356  4.205057  2.4188123
 3.1648052 4.076707  3.587126  3.2809978 4.426264  4.700568  5.9852467
 4.9791274 3.969011  4.0391316 4.5593505 4.307744  5.1371026 4.1603646
 3.509131  

In [8]:
from scipy.stats import pearsonr
cor_pearsonr=pearsonr(result,test_label)

In [9]:
print(cor_pearsonr)

(0.6272961312653618, 2.9536114762528334e-116)


In [10]:
model.save('./weight_MSE_N_0_1201_cnn3.h5')
model.save('./weight_MSE_N_20_1201_cnn5.h5')
model.save('./weight_MSE_R_0_1210_cnn1.h5')
model.save('./weight_MSE_R_20_1210_cnn1.h5')